In [1]:
from collections import namedtuple
import os


In [8]:
os.chdir('..')

In [9]:
%pwd

'd:\\Manav\\Projects\\cnnClassifier'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils import *

In [4]:
class ConfigurationManager:
    def __init__(self,
                config_filepath= CONFIG_FILE_PATH,
                params_filepath= PARAMS_FILE_PATH):

                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                
                create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [5]:
from urllib import request
from zipfile import ZipFile

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file)
            logger.info(f'{filename} downloaded with following info: \n{headers}')
        else:
            logger.info(f'File already exists with size : {get_size(Path(self.config.local_data_file))}')
    
    def _get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith('.jpg') and ('Cat' in f or 'Dog' in f)]
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_file_path = os.path.join(working_dir, f)
        if not os.path.exists(target_file_path):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_file_path) == 0:
            os.remove(target_file_path)
    
    def extract_zipfile(self):
        '''
        zip_file_path : str
        Extracts the zip file into data directory path
        Function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zip_ref:
            list_of_files = zip_ref.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zip_ref, f, unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2024-03-22 15:02:45,729: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-22 15:02:45,731: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-22 15:02:45,733: INFO: common: yaml file: schemas.yaml loaded successfully]


ValueError: yaml file is empty